In [1]:
from Models import *
from Utils import *
import warnings
# warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
from gensim.models import KeyedVectors
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load('/work/data/bpe/en.wiki.bpe.op50000.model')
bpe = KeyedVectors.load_word2vec_format("/work/data/bpe/en.wiki.bpe.op50000.d200.w2v.bin", binary=True)

In [3]:
df = pd.read_csv("/work/data/train_data/30M_QD.txt", nrows=100000, usecols=[0,1,2], names=["label","q", "d"], sep="\t", header=None, error_bad_lines=False)


In [72]:
bpe_dict = { bpe.index2word[i]: i for i in range(len(bpe.index2word))}

In [8]:
df_may, qrel_may = get_test_data("MayFlower")
df_june, qrel_june = get_test_data("JuneFlower")
df_july, qrel_july = get_test_data("JulyFlower")


b'Skipping line 19898: expected 6 fields, saw 8\nSkipping line 20620: expected 6 fields, saw 8\nSkipping line 38039: expected 6 fields, saw 8\n'


In [13]:
max_len = 10
enablePadding = True

q_may = parse_texts_bpe(df_may.q.tolist(), sp, bpe_dict, max_len, enablePadding)
d_may = parse_texts_bpe(df_may.d.tolist(), sp, bpe_dict, max_len, enablePadding)

q_june = parse_texts_bpe(df_june.q.tolist(), sp, bpe_dict, max_len, enablePadding)
d_june = parse_texts_bpe(df_june.d.tolist(), sp, bpe_dict, max_len, enablePadding)


q_july = parse_texts_bpe(df_july.q.tolist(), sp, bpe_dict, max_len, enablePadding)
d_july = parse_texts_bpe(df_july.d.tolist(), sp, bpe_dict, max_len, enablePadding)



In [94]:
count = 0
for i in q_june:
    for j in i:
        if j == bpe_dict['<unk>']:
            count = count + 1


In [95]:
np.sum([len(i) for i in q_june])

397970

In [96]:
count

2736

In [97]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential

w2v = Sequential()
w2v.add(bpe.get_keras_embedding(train_embeddings=False))
w2v.add(GlobalAveragePooling1D())

pred = get_cosine_sim(w2v.predict(q_july), w2v.predict(d_july))


In [98]:
auc(df_july.label.values, pred)

0.5321666036602992

In [3]:
%%time
count = 0
file = open("/work/workspace/Texygen/data/1M_Query_GAN", "a+") 
with open ("/data/chzho/deepqts/train_data/unifiedclick/join_oneyearsample_2B_training_all_top10", "r") as myfile:
    for line in myfile:
        s = line.split("\t")
        market = s[5]
        if "en-" not in market:
            continue
        
#         re.sub(r'\W+', ' ', s[1].lower())
#         file.write("%s\t%s\t%s\n" % (s[0], re.sub(r'\W+', ' ', s[1].lower()), re.sub(r'\W+', ' ', s[3].lower())))

        count = count + 1

        if count > 1000000:
            break
        break
file.close()

CPU times: user 22min 23s, sys: 1min 22s, total: 23min 46s
Wall time: 29min 14s


In [4]:
import keras
import numpy as np

model = keras.Sequential()
model.add(keras.layers.Dense(10, input_shape=(5, 6)))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy')

x_data = np.random.random((32, 5, 6))
y_data = np.random.randint(0, 9, size=(32,5,1))

model.fit(x=x_data, y=y_data, batch_size=16, epochs=3)

Epoch 1/3
32/32 [==============================] - 1s 23ms/step - loss: 11.5255
Epoch 2/3
32/32 [==============================] - 0s 182us/step - loss: 11.4506
Epoch 3/3
32/32 [==============================] - 0s 190us/step - loss: 11.4304


In [68]:
%%time
count = 0
file = open("/work/data/train_data/QueryQueryLog", "a+") 
query = []
with open ("/work/data/train_data/1M_EN_QQ_log", "r") as myfile:
    for line in myfile:
        s = line.replace("\n", "").split("\t")
#         print(line)
        if len(s) < 2:
            continue
        tmp = s[1].split("<sep>")
        
        __ = re.sub(r'\W+', ' ', s[0].lower())
        for i in tmp:
            _ = re.sub(r'\W+', ' ', i.lower())
            
            file.write(__+"\t"+_+"\t1\n")
        
file.close()

CPU times: user 27 s, sys: 376 ms, total: 27.4 s
Wall time: 27.6 s


In [ ]:
%%time
count = 0
file = open("/work/data/train_data/QueryLogBPE", "a+") 
query = []
with open ("/work/data/train_data/1M_EN_QQ_log", "r") as myfile:
    for line in myfile:
        s = line.replace("\n", "").split("\t")
#         print(line)
        if len(s) < 2:
            continue
        tmp = s[1].split("<sep>")
        
        __ = re.sub(r'\W+', ' ', s[0].lower())
        for i in tmp:
            _ = re.sub(r'\W+', ' ', i.lower())
            
            file.write(__+"\t"+_+"\t1\n")
        
file.close()

In [42]:
df = pd.read_csv("/work/data/train_data/QueryLog", names=["q"], sep="\t", header=None, error_bad_lines=False)


In [65]:
%%time
df_bpe_token = set([])
for i in df.q.tolist():
    for j in sp.EncodeAsPieces(i):
        df_bpe_token.add(j)


CPU times: user 1min 41s, sys: 696 ms, total: 1min 42s
Wall time: 1min 41s


In [124]:
len(df_token)

561556

In [45]:
df_token = set([])
july_token = set([])

for i in df.q.tolist():
    for j in i.split():
        df_token.add(j)


In [47]:
df_may, qrel_may = get_test_data("MayFlower", "/work/")
df_june, qrel_june = get_test_data("JuneFlower", "/work/")
df_july, qrel_july = get_test_data("JulyFlower", "/work/")

b'Skipping line 19898: expected 6 fields, saw 8\nSkipping line 20620: expected 6 fields, saw 8\nSkipping line 38039: expected 6 fields, saw 8\n'


In [64]:
print("Term Overlap between train and test dataset. QueryLog")
for a in [df_may, df_june, df_july]:
    token = set([])
    for i in a.q.tolist() + a.d.tolist():
        for j in i.split():
            token.add(j)
    print(len(token))
    print(len(df_token.intersection(token)))

Term Overlap between train and test dataset. QueryLog
12853
9648
13351
8799
1372
1310


In [66]:
print("BPE tokens overlap between train and test dataset. QueryLog")
for a in [df_may, df_june, df_july]:
    token = set([])
    for i in a.q.tolist() + a.d.tolist():
        for j in sp.EncodeAsPieces(i):
            token.add(j)
    print(len(token))
    print(len(df_bpe_token.intersection(token)))

BPE tokens overlap between train and test dataset. QueryLog
11276
10734
10280
10162
1556
1525


In [69]:
df = pd.read_csv("/work/data/train_data/QueryLog", names=["q"], sep="\t", header=None, error_bad_lines=False)

In [74]:
%%time
q_df = parse_texts_bpe(df.q.tolist(), sp, bpe_dict, 5, enablePadding=True)


CPU times: user 2min 40s, sys: 1.76 s, total: 2min 42s
Wall time: 2min 40s


In [95]:
x_test_reconstructed = np.random.randint(5, size=(1,5, 5))
sent_idx = 0
reconstructed_indexes = np.apply_along_axis(np.argmax, 1, x_test_reconstructed[sent_idx])


In [96]:
reconstructed_indexes

array([4, 2, 2, 2, 0])

In [97]:
x_test_reconstructed

array([[[3, 0, 0, 2, 4],
        [2, 3, 4, 1, 1],
        [1, 1, 2, 0, 2],
        [0, 2, 4, 2, 1],
        [4, 0, 3, 0, 4]]])

In [90]:
np.apply_along_axis?

In [119]:
a = np.random.randint(10, size=(1,4, 3))
gen_y = np.argmax(a, axis=-1)


In [120]:
gen_y

array([[1, 0, 1, 0]])

In [121]:
np.apply_along_axis(np.argmax, 1, a[0])

array([1, 0, 1, 0])

In [122]:
a

array([[[2, 7, 6],
        [7, 5, 4],
        [1, 3, 0],
        [9, 5, 5]]])